In [1]:
# Import packages
from Diff import Diff
%run startup.py
# Load all questions
load_questions()

ModuleNotFoundError: No module named 'Diff'

In [2]:
# initialize network and snapshot
NETWORK_NAME = "forwarding-change-validation"
BASE_NAME = "base"
BASE_PATH = "networks/forwarding-change-validation/base"

bf_set_network(NETWORK_NAME)
# bf_init_snapshot(BASE_PATH, name=BASE_NAME, overwrite=True)
bf_set_snapshot(name = BASE_NAME)

'base'

In [3]:
df = bfq.routes().answer(snapshot=BASE_NAME).frame()

In [6]:
df[(df['Node']=='border1')&(df['Protocol']=='local')]

,Node,VRF,Network,Next_Hop,Next_Hop_IP,Next_Hop_Interface,Protocol,Metric,Admin_Distance,Tag
1,border1,default,10.12.11.2/32,None,AUTO/NONE(-1l),GigabitEthernet0/0,local,0,0,None
127,border1,default,2.12.11.1/32,None,AUTO/NONE(-1l),GigabitEthernet1/0,local,0,0,None
154,border1,default,2.12.12.1/32,None,AUTO/NONE(-1l),GigabitEthernet2/0,local,0,0,None


In [6]:
df[(df['Network']=='2.1.1.1/32')&(df['Protocol']=='connected')]

,Node,VRF,Network,Next_Hop,Next_Hop_IP,Next_Hop_Interface,Protocol,Metric,Admin_Distance,Tag
21,border1,default,2.1.1.1/32,None,AUTO/NONE(-1l),Loopback0,connected,0,0,None


In [24]:
result = bfq.detectLoops().answer(snapshot=BASE_NAME).frame()
show(result)

,Flow,Traces,TraceCount


In [5]:
# tr_frame = bfq.traceroute(startLocation="border1", headers=HeaderConstraints(dstIps="spine2")).answer().frame()
# Display results using customizations to handle large string values
# show(tr_frame)

In [6]:
# invoke bfq.routes to push batfish parse config files and compute data plane
# base_routes = bfq.routes().answer(snapshot = BASE_NAME).frame()
# connected_df = base_routes[(base_routes["Protocol"]=="connected")][["Node","Network","Next_Hop_Interface"]]
# grouped_df = connected_df.groupby(connected_df["Network"])
# for network,group_data in grouped_df:
#     if(group_data.Node.count()==1):
#         continue)
#     node1 = group_data.iloc[0,0]
#     intf1 = group_data.iloc[0,2]
#     node2 = group_data.iloc[1,0]
#     intf2 = group_data.iloc[1,2]

#     line = "{:15} {:25} {:15} {:25}".format(node1,intf1,node2,intf2)
#     print(line)

In [4]:
CHANGE_NAME = "change"
CHANGE_PATH = "networks/forwarding-change-validation/change1"
bf_set_snapshot(name=CHANGE_NAME)
# bf_init_snapshot(CHANGE_PATH,name = CHANGE_NAME, overwrite=True)

'change'

In [5]:
# invoke bfq.routes to push batfish parse config files and compute data plane
change_routes = bfq.routes().answer(snapshot = CHANGE_NAME).frame()

In [26]:
result = bfq.detectLoops().answer(snapshot=CHANGE_NAME).frame()
show(result)

,Flow,Traces,TraceCount
0,Start Location: border2Src IP: 2.1.1.2Dst IP: 10.12.11.2IP Protocol: ICMP,"LOOP1. node: border2 ORIGINATED(default) FORWARDED(Routes: ospfE2 [Network: 10.12.11.0/24, Next Hop IP:2.12.22.2]) TRANSMITTED(GigabitEthernet1/0)2. node: core2 RECEIVED(GigabitEthernet0/0) FORWARDED(Routes: static [Network: 10.12.11.2/32, Next Hop IP:2.23.22.3]) TRANSMITTED(GigabitEthernet2/0)3. node: spine2 RECEIVED(GigabitEthernet0/0) FORWARDED(Routes: static [Network: 10.12.11.2/32, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet2/0)4. node: leaf1 RECEIVED(GigabitEthernet1/0) PERMITTED(RESTRICT_NETWORK_TRAFFIC_IN (INGRESS_FILTER)) FORWARDED(Routes: static [Network: 10.12.11.2/32, Next Hop IP:2.34.101.3]) TRANSMITTED(GigabitEthernet0/0)5. node: spine1 RECEIVED(GigabitEthernet2/0) FORWARDED(Routes: static [Network: 10.12.11.2/32, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet0/0)6. node: core1 RECEIVED(GigabitEthernet2/0) FORWARDED(Routes: ospfE2 [Network: 10.12.11.0/24, Next Hop IP:2.12.21.1]) TRANSMITTED(GigabitEthernet1/0)7. node: border2 RECEIVED(GigabitEthernet2/0)",1


In [17]:
from client.apkeep import APKeep

In [18]:
# Initialize an apkeep model based on the original configuration
ap = APKeep('forwarding-change-validation','base')

In [19]:
ans1 = ap.base_check()

loops : 0


In [20]:
ans2 = ap.update_check('change')

loops : 1


In [21]:
ans2.loops()

,aps_info,loop_path,loop_num
0,3671 : 10.12.11.2/323819 : 10.12.11.2/32,"[core2,GigabitEthernet2/0, spine2,GigabitEthernet0/0, spine2,GigabitEthernet2/0, leaf1_RESTRICT_NETWORK_TRAFFIC_IN_GigabitEthernet1/0_in,inport, leaf1_RESTRICT_NETWORK_TRAFFIC_IN_GigabitEthernet1/0_in,permit, leaf1,GigabitEthernet1/0, leaf1,GigabitEthernet0/0, spine1,GigabitEthernet2/0, spine1,GigabitEthernet1/0, core2,GigabitEthernet3/0, core2,GigabitEthernet2/0]",1
